In [35]:
import os
import shutil
from shutil import copyfile
import pandas as pd
import csv
import cv2

import xml.etree.ElementTree as ET

In [12]:
dataFrame = pd.read_csv('allAnnotations.csv')
print(dataFrame.shape)

(7855, 12)


In [13]:
def create_xml(filename, img_size, category, bound_box):
    annotation = ET.Element("annotation")
    ET.SubElement(annotation, "filename").text=filename
    
    size = ET.SubElement(annotation, "size")
    ET.SubElement(size, "width").text = str(img_size[0])
    ET.SubElement(size, "height").text = str(img_size[1])

    #num_object = ET.SubElement(annotation, "num_object").text=str(1)
    object1 = ET.SubElement(annotation, "object")
    ET.SubElement(object1, "name").text = category
    bndbox = ET.SubElement(object1, "bndbox")
    ET.SubElement(bndbox, "xmax").text = str(bound_box[0])
    ET.SubElement(bndbox, "xmin").text = str(bound_box[1])
    ET.SubElement(bndbox, "ymax").text = str(bound_box[2])
    ET.SubElement(bndbox, "ymin").text = str(bound_box[3])

    tree = ET.ElementTree(annotation)
    tree.write(filename+'.xml')
    
def insert_xml(filename, category, bound_box):
    annotation = ET.parse(filename+'.xml').getroot()
    object_new = ET.SubElement(annotation, 'object')
    ET.SubElement(object_new, "name").text = category
    bndbox = ET.SubElement(object_new, "bndbox")
    ET.SubElement(bndbox, "xmax").text = str(bound_box[0])
    ET.SubElement(bndbox, "xmin").text = str(bound_box[1])
    ET.SubElement(bndbox, "ymax").text = str(bound_box[2])
    ET.SubElement(bndbox, "ymin").text = str(bound_box[3])
    tree = ET.ElementTree(annotation)
    tree.write(filename+'.xml')

In [43]:
DEBUG = False
if DEBUG:
    i = 0

img_list = list()

for index,row in dataFrame.iterrows():
    if DEBUG:
        if i == 3:
            break
    img_name = row['Filename']
    if DEBUG:
        print("image name:")
        print(img_name)

    img = cv2.imread(img_name) # BGR image
        
    img_size = img.shape[:2]
    img_cate = row['Annotation tag']
    img_bbox = [row['Upper left corner X'],row['Lower right corner X'],
                row['Upper left corner Y'],row['Lower right corner Y']]
    xml_name = img_name.split('.png')[0].split('/')[-1]
    
    if xml_name in img_list:
        insert_xml(xml_name, img_cate, img_bbox)
    else:
        create_xml(xml_name, img_size, img_cate, img_bbox)
    
    img_copy = 'cp %s ./JPEGImages' % img_name
    img_move = 'mv %s.xml ./Annotations' % img_name
    img_list.append(xml_name)
    
    if DEBUG:
        print("img_list:")
        print(img_list)
        
    os.system(img_copy)
    os.system(xml_move)
    if DEBUG:
        i = i + 1

image name:
aiua120214-0/frameAnnotations-DataLog02142012_external_camera.avi_annotations/stop_1330545910.avi_image0.png
img_list:
['stop_1330545910.avi_image0']
image name:
aiua120214-0/frameAnnotations-DataLog02142012_external_camera.avi_annotations/stop_1330545910.avi_image0.png
img_list:
['stop_1330545910.avi_image0', 'stop_1330545910.avi_image0']
image name:
aiua120214-0/frameAnnotations-DataLog02142012_external_camera.avi_annotations/stop_1330545910.avi_image1.png
img_list:
['stop_1330545910.avi_image0', 'stop_1330545910.avi_image0', 'stop_1330545910.avi_image1']
